In [1]:
!pip install lightfm

In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from lightfm import LightFM

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

from sklearn.metrics import ndcg_score

import warnings
warnings.filterwarnings('ignore')

In [3]:
seed = 228
torch.manual_seed(seed)
np.random.seed(seed)

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Выбрал датасет movielens датасет, так как мы уже работали с ним в первой домашке.

Он достаточно маленький, чтобы методы учились на нём не по три часа, но и достаточно большой, чтобы не было так, что три с половиной наблюдения.

К тому же в нём нет кучи лишних данных, которые нужно думать как использовать: всего лишь инфа о просмотрах и оценках.

In [5]:
prefix = '/content/drive/My Drive/recsys-hw3/'

In [6]:
ratings = pd.read_csv(prefix + 'ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')
movie_info = pd.read_csv(prefix + 'movies.dat', delimiter='::', header=None, names=['movie_id', 'name', 'category'], engine='python')

ratings = ratings.sort_values(by = ['user_id', 'movie_id'])
movie_info = movie_info.sort_values(by = ['movie_id'])

In [7]:
ratings.head()

,user_id,movie_id,rating
40,1,1,5
25,1,48,5
39,1,150,5
44,1,260,4
23,1,527,5


In [8]:
movie_info.head()

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]
implicit_ratings['rating'] = 1

In [10]:
implicit_ratings.head(10)

,user_id,movie_id,rating
40,1,1,1
25,1,48,1
39,1,150,1
44,1,260,1
23,1,527,1
49,1,531,1
33,1,588,1
8,1,594,1
10,1,595,1
51,1,608,1


In [11]:
all_users = np.unique(implicit_ratings["user_id"])
users_old2new = {old_i: new_i + 1 for new_i, old_i in enumerate(all_users)}

all_movies = np.unique(movie_info["movie_id"])
movies_old2new = {old_i: new_i + 1 for new_i, old_i in enumerate(all_movies)}

movie_info['movie_id'] = np.arange(len(all_movies)) + 1

implicit_ratings['user_id'] = implicit_ratings['user_id'].apply(lambda x: users_old2new[x])
implicit_ratings['movie_id'] = implicit_ratings['movie_id'].apply(lambda x: movies_old2new[x]) 

In [12]:
movie_info.head(10)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [13]:
implicit_ratings.head(10)

,user_id,movie_id,rating
40,1,1,1
25,1,48,1
39,1,149,1
44,1,258,1
23,1,524,1
49,1,528,1
33,1,585,1
8,1,591,1
10,1,592,1
51,1,605,1


Сделаем тестовый сет с помощью идеи leave one out.

In [14]:
seen_by_user = {}
unseen_by_user = {}

train = np.empty((0, 2), int)
test = np.empty((0, 2), int)

for i in range(1, len(all_users) + 1):
    cur_seen_movies = implicit_ratings[implicit_ratings['user_id'] == i]
    seen_by_user[i] = set(cur_seen_movies['movie_id'])

    cur_seen_movies = np.array(cur_seen_movies)[:, :2]

    unseen_by_user[i] = set(np.arange(1, len(all_movies) + 1)) - seen_by_user[i]

    seen_by_user[i] = np.array(list(seen_by_user[i]))
    unseen_by_user[i] = np.array(list(unseen_by_user[i]))

    if len(seen_by_user[i]) == 1:
        print('Bad one!')
        continue 
           
    loo = np.random.randint(0, len(cur_seen_movies))
    train = np.vstack((train, cur_seen_movies[:loo], cur_seen_movies[loo + 1:]))
    test = np.vstack((test, cur_seen_movies[loo]))
        

Bad one!


In [15]:
get_similars = lambda item_id, model : [(movie_info[movie_info["movie_id"] == x[0]]["name"].to_string(), x[1])
                                        for x in model.similar_items(item_id)]

get_recommendations = lambda user_id,  model: [(movie_info[movie_info["movie_id"] == x[0]]["name"].to_string(), x[1]) 
                                               for x in model.recommendation(user_id)]

get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

Тестировать качество модели будем подмешивая к одному ГАРАНТИРОВАННО просмотренному фильму 99 случайных непросмотренных.

In [16]:
def eval_metrics(model, test, k=10):
    labels = np.zeros(100)
    labels[0] = 1 
    ndcg = []
    recall = []
    
    for cur_line in test:
        cur_unseen = np.full((99, 2), cur_line[0])
        cur_unseen[:, 1] = np.random.choice(unseen_by_user[cur_line[0]], 99, replace=False)
        input = np.vstack((cur_line, cur_unseen))

        pred = model.predict(input)

        ndcg.append(ndcg_score([labels], [pred], k=10))
        recall.append(np.sum(np.argsort(pred)[-k:] == 0))

    ndcg = np.array(ndcg)
    recall = np.array(recall)

    mean_ndcg = np.mean(ndcg)
    mean_recall = np.mean(recall)

    print(f"NDCG@{k} = {mean_ndcg}")
    print(f"RECALL@{k} = {mean_recall}")

    return mean_ndcg, mean_recall

# WARP

In [17]:
train_sparse = sp.coo_matrix((np.ones_like(train[:, 0]), (train[:, 0], train[:, 1])), shape=(len(all_users) + 1, len(all_movies) + 1))

train_sparse = train_sparse.tocsr()

In [18]:
warp = LightFM(no_components = 100,
                loss='warp',
                random_state=seed)

warp.fit(train_sparse, epochs=50, verbose=True, num_threads=4)

Epoch: 100%|██████████| 50/50 [00:52<00:00,  1.04s/it]


In [19]:
class warp_wrapper():
    def __init__(self, warp):
        self.model = warp
        self.item_embed = warp.item_embeddings

        self.item_embed /= np.linalg.norm(self.item_embed, axis=-1).reshape(-1, 1)
    
    def similar_items(self, item_id, k=10):
        scores = self.item_embed @ self.item_embed[item_id]
        best = np.argpartition(scores, -k)[-k:]
        return sorted(zip(best, scores[best]), key=lambda x: x[1], reverse=True)

    def recommendation(self, user_id, k=10):
        scores = self.model.predict(user_id, unseen_by_user[user_id])
        
        return sorted(zip(unseen_by_user[user_id], scores), key=lambda x: x[1], reverse=True)[:k]
    
    def predict(self, test):
        return self.model.predict(test[:, 0], test[:, 1])

In [20]:
warp_wrapped = warp_wrapper(warp)

In [21]:
get_similars(1, warp_wrapped)

[('0    Toy Story (1995)', 1.0000001),
 ('3045    Toy Story 2 (1999)', 0.6473875),
 ("2286    Bug's Life, A (1998)", 0.55438536),
 ('584    Aladdin (1992)', 0.4951325),
 ('591    Beauty and the Beast (1991)', 0.49364695),
 ('360    Lion King, The (1994)', 0.489245),
 ('2225    Antz (1998)', 0.47896263),
 ('33    Babe (1995)', 0.47846648),
 ('1838    Mulan (1998)', 0.46582747),
 ('547    Nightmare Before Christmas, The (1993)', 0.46215713)]

In [22]:
get_recommendations(4, warp_wrapped)

[('847    Godfather, The (1972)', 1.3981317),
 ("523    Schindler's List (1993)", 1.2223358),
 ('589    Silence of the Lambs, The (1991)', 1.2113236),
 ('585    Terminator 2: Judgment Day (1991)', 1.1630515),
 ('1178    Star Wars: Episode V - The Empire Strikes Back...', 1.1519053),
 ('108    Braveheart (1995)', 1.0510399),
 ('2789    American Beauty (1999)', 1.0402995),
 ('1284    Butch Cassidy and the Sundance Kid (1969)', 0.96933603),
 ("1176    One Flew Over the Cuckoo's Nest (1975)", 0.9516682),
 ('1192    Star Wars: Episode VI - Return of the Jedi (1983)', 0.93168443)]

In [23]:
get_user_history(4, implicit_ratings)

['257    Star Wars: Episode IV - A New Hope (1977)',
 '476    Jurassic Park (1993)',
 '1023    Die Hard (1988)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '1180    Raiders of the Lost Ark (1981)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1196    Alien (1979)',
 '1220    Terminator, The (1984)',
 '1366    Jaws (1975)',
 '1885    Rocky (1976)',
 '1959    Saving Private Ryan (1998)',
 '2297    King Kong (1933)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '2882    Fistful of Dollars, A (1964)',
 '3349    Thelma & Louise (1991)',
 '3399    Hustler, The (1961)',
 '3633    Mad Max (1979)']

In [24]:
warp_res = eval_metrics(warp_wrapped, test)

NDCG@10 = 0.5089535323442063
RECALL@10 = 0.7808514162663575


warp из коробки показывает уже неплохое качество, пробовал тюнить параметры, но заметного прироста это не дало, а часто только делало хуже, поэтому оставил стандартный вариант.

# NCF

Предобучим две сетки mlp и gml, чтобы использовать их в методе ncf.
По дороге можно замерить качество для каждой сетки по отдельности.

В комментариях оставил параметры оптимайзера, с которыми я обучал и количество эпох.

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [26]:
class MLP(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100, hidden_size=128):
        super(MLP, self).__init__()
        self.embedding_user = nn.Embedding(num_embeddings=num_users, embedding_dim=emb_size)
        self.embedding_item = nn.Embedding(num_embeddings=num_items, embedding_dim=emb_size)
        
        self.model = nn.Sequential(nn.Linear(2 * emb_size, hidden_size), 
                                   nn.ReLU(), 
                                   nn.Linear(hidden_size, hidden_size),
                                   nn.ReLU(), 
                                   nn.Linear(hidden_size, hidden_size),
                                   nn.ReLU(), 
                                   nn.Linear(hidden_size, hidden_size), 
                                   nn.ReLU())

        self.last =  nn.Sequential(nn.Linear(hidden_size, 1), 
                                   nn.Sigmoid())
        
        self.lossfn = nn.BCELoss()
        
    def forward(self, user_id, item_id):
        output = torch.cat([self.embedding_user(user_id), self.embedding_item(item_id)], dim=-1)
        output = self.model(output)
        score = self.last(output)

        return score
    
    def part_forward(self, user_id, item_id):
        output = torch.cat([self.embedding_user(user_id), self.embedding_item(item_id)], dim=-1)
        output = self.model(output)
            
        return output

    def loss(self, user_id, item_id, target):
        pred = self.forward(user_id, item_id)

        return self.lossfn(pred, target.reshape(-1, 1))

    def predict(self, test):
        res = []
        dataloader = torch.utils.data.DataLoader(test, batch_size=2048)
        for batch in dataloader:
            pred = self.forward(batch[:, 0].to(device), batch[:, 1].to(device))
            res.append(pred.squeeze().cpu().detach().numpy())
        return np.hstack(res)
    
    def create_embeddings(self):
        self.item_embed = self.embedding_item.weight.detach().cpu().numpy()
        self.item_embed /= np.linalg.norm(self.item_embed, axis=-1).reshape(-1, 1)
    
    def similar_items(self, item_id, k=10):
        scores = self.item_embed @ self.item_embed[item_id]
        best = np.argpartition(scores, -k)[-k:]
        return sorted(zip(best, scores[best]), key=lambda x: x[1], reverse=True)

    def recommendation(self, user_id, k=10):
        cur_unseen = unseen_by_user[user_id]
        scores = self.predict(np.concatenate([np.array([user_id] * len(cur_unseen)).reshape(-1, 1), cur_unseen.reshape(-1, 1)], 1))

        return sorted(zip(cur_unseen, scores), key=lambda x: x[1], reverse=True)[:k]

In [27]:
class my_dataset:
    def __init__(self, dataset):
        self.dataset = dataset
        self.unique_users = np.unique(dataset[:, 0])

    def __getitem__(self, index):
        if index < len(self.dataset):
            return self.dataset[index, 0], self.dataset[index, 1], 1
            
        user = np.random.choice(self.unique_users, 1)[0]
        item = np.random.choice(unseen_by_user[user], 1)[0]
        return user, item, 0

    def __len__(self):
        return 2 * len(self.dataset)

In [28]:
train_data = my_dataset(train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=2048, shuffle=True)

In [29]:
def train_model(model, train_loader, optimizer, epochs=50, log_freq=5, name='mlp.pkl'):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for user_id, item_id, target in train_loader:
            loss = model.loss(user_id.to(device), item_id.to(device), target.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if epoch % log_freq == 0:
            print(f'epoch #{epoch}, train loss :{total_loss / len(train_loader)}')

    model.create_embeddings()
    torch.save(model.state_dict(), prefix + name)

In [30]:
num_items = np.max(implicit_ratings["movie_id"]) + 1
num_users = np.max(implicit_ratings['user_id']) + 1

In [31]:
mlp = MLP(num_users, num_items, 100, hidden_size=128).to(device)

mlp.load_state_dict(torch.load(prefix + 'mlp.pkl'))
mlp.create_embeddings()

#optimizer = optim.Adam(mlp.parameters(), 1e-2)
#train_model(mlp, train_loader, optimizer, epochs=50)

In [32]:
get_similars(1, mlp)

[('0    Toy Story (1995)', 0.9999998),
 ('3045    Toy Story 2 (1999)', 0.7747952),
 ('1245    Groundhog Day (1993)', 0.7050154),
 ('33    Babe (1995)', 0.69508135),
 ('2327    Shakespeare in Love (1998)', 0.6687774),
 ('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.64759105),
 ('584    Aladdin (1992)', 0.6433375),
 ('2255    Life Is Beautiful (La Vita � bella) (1997)', 0.6344376),
 ('257    Star Wars: Episode IV - A New Hope (1977)', 0.6338364),
 ('591    Beauty and the Beast (1991)', 0.6279571)]

In [33]:
get_recommendations(4, mlp)

[('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.9903384),
 ('585    Terminator 2: Judgment Day (1991)', 0.98567957),
 ('847    Godfather, The (1972)', 0.9853867),
 ('2502    Matrix, The (1999)', 0.9830921),
 ('1192    Star Wars: Episode VI - Return of the Jedi (1983)', 0.9804739),
 ('2879    From Russia with Love (1963)', 0.9722955),
 ('1203    Godfather: Part II, The (1974)', 0.96999633),
 ('2847    Total Recall (1990)', 0.96996796),
 ('108    Braveheart (1995)', 0.9593628),
 ('1182    Aliens (1986)', 0.9563243)]

In [34]:
mlp_res = eval_metrics(mlp, test)

NDCG@10 = 0.6756500430933812
RECALL@10 = 0.9425211197614709


In [35]:
class GMF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(GMF, self).__init__()
        self.embedding_user = nn.Embedding(num_embeddings=num_users, embedding_dim=emb_size)
        self.embedding_item = nn.Embedding(num_embeddings=num_items, embedding_dim=emb_size)

        self.last =  nn.Sequential(nn.Linear(emb_size, 1), 
                                   nn.Sigmoid())
        
        self.lossfn = nn.BCELoss()
        
    def forward(self, user_id, item_id):
        output = torch.mul(self.embedding_user(user_id), self.embedding_item(item_id))
        score = self.last(output)

        return score
    
    def part_forward(self, user_id, item_id):
        output = torch.mul(self.embedding_user(user_id), self.embedding_item(item_id))
            
        return output
    
    def loss(self, user_id, item_id, target):
        pred = self.forward(user_id, item_id)

        return self.lossfn(pred, target.reshape(-1, 1))
    
    def predict(self, test):
        res = []
        dataloader = torch.utils.data.DataLoader(test, batch_size=2048)
        for batch in dataloader:
            pred = self.forward(batch[:, 0].to(device), batch[:, 1].to(device))
            res.append(pred.squeeze().cpu().detach().numpy())
        return np.hstack(res)
    
    def create_embeddings(self):
        self.item_embed = self.embedding_item.weight.detach().cpu().numpy()
        self.item_embed /= np.linalg.norm(self.item_embed, axis=-1).reshape(-1, 1)
    
    def similar_items(self, item_id, k=10):
        scores = self.item_embed @ self.item_embed[item_id]
        best = np.argpartition(scores, -k)[-k:]
        return sorted(zip(best, scores[best]), key=lambda x: x[1], reverse=True)

    def recommendation(self, user_id, k=10):
        cur_unseen = unseen_by_user[user_id]
        scores = self.predict(np.concatenate([np.array([user_id] * len(cur_unseen)).reshape(-1, 1), cur_unseen.reshape(-1, 1)], 1))

        return sorted(zip(cur_unseen, scores), key=lambda x: x[1], reverse=True)[:k]

In [36]:
gmf = GMF(num_users, num_items, 100).to(device)

gmf.load_state_dict(torch.load(prefix + 'gmf.pkl'))
gmf.create_embeddings()

#optimizer = optim.Adam(gmf.parameters(), 1e-2)
#train_model(gmf, train_loader, optimizer, epochs=10, name='gmf.pkl')

In [37]:
get_similars(1, gmf)

[('0    Toy Story (1995)', 1.0),
 ('3045    Toy Story 2 (1999)', 0.6507826),
 ('33    Babe (1995)', 0.46679562),
 ('584    Aladdin (1992)', 0.46370733),
 ("2286    Bug's Life, A (1998)", 0.4512086),
 ('1132    Wrong Trousers, The (1993)', 0.4030059),
 ('1838    Mulan (1998)', 0.3961269),
 ('2009    Jungle Book, The (1967)', 0.38822633),
 ('360    Lion King, The (1994)', 0.3867544),
 ('711    Wallace & Gromit: The Best of Aardman Animatio...', 0.38291404)]

In [38]:
get_recommendations(4, gmf)

[('585    Terminator 2: Judgment Day (1991)', 0.9993358),
 ('847    Godfather, The (1972)', 0.989917),
 ('1203    Godfather: Part II, The (1974)', 0.9876106),
 ('2879    From Russia with Love (1963)', 0.98425853),
 ("1176    One Flew Over the Cuckoo's Nest (1975)", 0.9780659),
 ('3585    Guns of Navarone, The (1961)', 0.97706884),
 ('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.96342677),
 ('1884    French Connection, The (1971)', 0.9544666),
 ('1284    Butch Cassidy and the Sundance Kid (1969)', 0.948415),
 ('108    Braveheart (1995)', 0.944661)]

In [39]:
gmf_res = eval_metrics(gmf, test)

NDCG@10 = 0.8225001261129232
RECALL@10 = 0.9884048368394898


In [40]:
class NCF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100, layer_size=128, mlp_name='mlp.pkl', gmf_name='gmf.pkl'):
        super(NCF, self).__init__()

        self.mlp = MLP(num_users, num_items, emb_size, layer_size)
        self.gmf = GMF(num_users, num_items, emb_size)

        self.mlp.load_state_dict(torch.load(prefix + mlp_name))
        self.gmf.load_state_dict(torch.load(prefix + gmf_name))

        self.last =  nn.Sequential(nn.Linear(emb_size + layer_size, 1), 
                                   nn.Sigmoid())
        
        self.lossfn = nn.BCELoss()
        
    def forward(self, user_id, item_id):
        mlp_out = self.mlp.part_forward(user_id, item_id)
        gmf_out = self.gmf.part_forward(user_id, item_id)

        output = torch.cat((mlp_out, gmf_out), dim=-1)
        score = self.last(output)

        return score
    
    def loss(self, user_id, item_id, target):
        pred = self.forward(user_id, item_id)

        return self.lossfn(pred, target.reshape(-1, 1))
    
    def predict(self, test):
        res = []
        dataloader = torch.utils.data.DataLoader(test, batch_size=2048)
        for batch in dataloader:
            pred = self.forward(batch[:, 0].to(device), batch[:, 1].to(device))
            res.append(pred.squeeze().cpu().detach().numpy())
        return np.hstack(res)
    
    def create_embeddings(self):
        self.item_embed = np.hstack((self.gmf.embedding_item.weight.detach().cpu().numpy(), self.mlp.embedding_item.weight.detach().cpu().numpy()))
        self.item_embed /= np.linalg.norm(self.item_embed, axis=-1).reshape(-1, 1)
    
    def similar_items(self, item_id, k=10):
        scores = self.item_embed @ self.item_embed[item_id]
        best = np.argpartition(scores, -k)[-k:]
        return sorted(zip(best, scores[best]), key=lambda x: x[1], reverse=True)

    def recommendation(self, user_id, k=10):
        cur_unseen = unseen_by_user[user_id]
        scores = self.predict(np.concatenate([np.array([user_id] * len(cur_unseen)).reshape(-1, 1), cur_unseen.reshape(-1, 1)], 1))

        return sorted(zip(cur_unseen, scores), key=lambda x: x[1], reverse=True)[:k]

С обучением ncf пришлось сильно шаманить, с lr = 0.01 лосс переставал падать уже после пары итераций и стопорился на 0.1, зато с lr = 0.001 моделька офигенно у меня училась часа два-три до лосса около 0.01 за 200 эпох, на чём я решил остановиться.

In [41]:
ncf = NCF(num_users, num_items, 100, 128).to(device)

ncf.load_state_dict(torch.load(prefix + 'ncf.pkl'))
ncf.create_embeddings()
#optimizer = optim.Adam(ncf.parameters(), 1e-3)
#train_model(ncf, train_loader, optimizer, epochs = 200, name='ncf.pkl')

In [42]:
get_similars(1, ncf)

[('0    Toy Story (1995)', 0.99999994),
 ('3045    Toy Story 2 (1999)', 0.75787383),
 ('1245    Groundhog Day (1993)', 0.654258),
 ('33    Babe (1995)', 0.6343057),
 ('2327    Shakespeare in Love (1998)', 0.6192112),
 ('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.6094531),
 ("2286    Bug's Life, A (1998)", 0.59257615),
 ('257    Star Wars: Episode IV - A New Hope (1977)', 0.5894151),
 ('584    Aladdin (1992)', 0.58114624),
 ('315    Shawshank Redemption, The (1994)', 0.57494915)]

In [43]:
get_recommendations(4, ncf)

[('585    Terminator 2: Judgment Day (1991)', 0.9996001),
 ('2879    From Russia with Love (1963)', 0.98286116),
 ('1178    Star Wars: Episode V - The Empire Strikes Back...', 0.98168576),
 ('3634    Mad Max 2 (a.k.a. The Road Warrior) (1981)', 0.96332955),
 ('847    Godfather, The (1972)', 0.9047079),
 ('108    Braveheart (1995)', 0.85878885),
 ('1267    Ben-Hur (1959)', 0.80412024),
 ('1182    Aliens (1986)', 0.6972705),
 ('1203    Godfather: Part II, The (1974)', 0.46086654),
 ('1284    Butch Cassidy and the Sundance Kid (1969)', 0.32009172)]

In [44]:
ncf_res = eval_metrics(ncf, test)

NDCG@10 = 0.8898115028772888
RECALL@10 = 0.9995030644359781


In [45]:
print(f'Results (NDCG@10 / RECALL@10):')
print(f'warp: {warp_res}')
print(f'only mlp: {mlp_res}')
print(f'only gmf: {gmf_res}')
print(f'ncf: {ncf_res}')

Results (NDCG@10 / RECALL@10):
warp: (0.5089535323442063, 0.7808514162663575)
only mlp: (0.6756500430933812, 0.9425211197614709)
only gmf: (0.8225001261129232, 0.9884048368394898)
ncf: (0.8898115028772888, 0.9995030644359781)


Видим, что mlp и gmf по отдельности получились слабее ncf-метода.

Все нейросетевые модели смогли победить warp, но это, скорее всего, из-за того, что у меня не получилось подобрать хорошие параметры для warp.

У всех моделей получились более-менее хорошие симилары. К мультику рекомендуются мультики. 

Рекомендации тоже неплохие. Это не просто популярные фильмы с высоким рейтингом, а именно блокбастеры, в частности, сиквелы просмотренных юзером фильмов.